In [ ]:
######     The sentence are converted into the 300-D vectors
######      Sentence_Vector = the gravity center of the distributed word vectors in the sentence.

######                          \Sigma (word2vec(iword) * \rho(iword)) 
######      sen_vec    = ----------------------------------
######                             (\Sigma \rho(iword))
######       In this version, \rho = 1/count(iword)  while tfidf would be more meaningful.

In [289]:
import gensim
import time
import numpy as np
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile
import re

In [290]:
# Step 1: Load the trained model: GoogleNews or Glove or whatever

since=time.time()
model = gensim.models.KeyedVectors.load_word2vec_format('/home/miaohan/trained_model/GoogleNews-vectors-negative300.bin', binary=True)

print(model['love'])
print("time:",time.time()-since)

[ 0.10302734 -0.15234375  0.02587891  0.16503906 -0.16503906  0.06689453
  0.29296875 -0.26367188 -0.140625    0.20117188 -0.02624512 -0.08203125
 -0.02770996 -0.04394531 -0.23535156  0.16992188  0.12890625  0.15722656
  0.00756836 -0.06982422 -0.03857422  0.07958984  0.22949219 -0.14355469
  0.16796875 -0.03515625  0.05517578  0.10693359  0.11181641 -0.16308594
 -0.11181641  0.13964844  0.01556396  0.12792969  0.15429688  0.07714844
  0.26171875  0.08642578 -0.02514648  0.33398438  0.18652344 -0.20996094
  0.07080078  0.02600098 -0.10644531 -0.10253906  0.12304688  0.04711914
  0.02209473  0.05834961 -0.10986328  0.14941406 -0.10693359  0.01556396
  0.08984375  0.11230469 -0.04370117 -0.11376953 -0.0037384  -0.01818848
  0.24316406  0.08447266 -0.07080078  0.18066406  0.03515625 -0.09667969
 -0.21972656 -0.00328064 -0.03198242  0.18457031  0.28515625 -0.0859375
 -0.11181641  0.0213623  -0.30664062 -0.09228516 -0.18945312  0.01513672
  0.18554688  0.34375    -0.31054688  0.22558594  0.

In [165]:
stop_words=['a','and','of','to','','\n']

In [209]:
#print(model['favourite'])

In [248]:
# Read the data into a list of strings.
def load_train(filename):
    text_id=[]
    text_content=[]
    text_author=[]
    tf_word=[]
    vocal=[]
    file=open(filename,"r")
    for line in file.readlines():
        #print("line",line)
        row=re.split(" |!|\?|\.|\,|\"|\'|;|\:", line)
        text_content=row[:]

        # strip the stop_words and meaningless str

        for word in row:  
            if word in stop_words:
                
                text_content.remove(word)
        #print(text_content)        
        
        
        text_id.append(text_content[0]) # load id
        #print("text_content[0]",text_content[0])
        del text_content[0]
 
        if text_content[-1] == "EAP":
            #print("EAP HERE!")
            text_author.append([1,0,0])
        elif text_content[-1] == "HPL":
            text_author.append([0,1,0])
        elif text_content[-1] == "MWS":
            text_author.append([0,0,1])   #load Author
            
        #print("text_content[-1]",text_content[-1])
        del text_content[-1]

        

                
        vec_sen_line=[0]*300     
        mass_sum=0    
        for word in text_content:        
            try:
                count_word=text_content.count(word)
                vec_sen_line=vec_sen_line+np.asarray(model[word]) / count_word  # center of gravity= ({Sigma pos(i)*mass(i)}/{Sigma mass(i)})
                mass_sum=mass_sum+1./count_word
            except:
                pass
                #print("UKN word",word)
                
        #print("vec_sen_line type",type(vec_sen_line))
        tf_word.append(np.asarray(vec_sen_line)/mass_sum)
     
    #print(text_id[0],text_author[0],tf_word[0])
    #print(len(text_id),len(text_author),len(tf_word))
    del text_id[0]
    del tf_word[0]
    print("file loaded!")
    file.close()
    return text_id,text_author,tf_word



since=time.time()
#filename='/home/miaohan/NLP/data/test.txt'
filename='/home/miaohan/NLP/data/train.csv'
text_id,text_author,tf_word=load_train(filename)
#text_id[-10:]
#text_content[-10:]


file loaded!


In [250]:
def load_test(filename):
    text_id=[]
    text_content=[]
    tf_word=[]
    
    file=open(filename,"r")
    for line in file.readlines():
        #print("line",line)
        row=re.split(" |!|\?|\.|\,|\"|\'|;|\:", line)
        text_content=row[:]

        # strip the stop_words and meaningless str

        for word in row:  
            if word in stop_words:
                
                text_content.remove(word)
        #print(text_content)        
        
        
        text_id.append(text_content[0]) # load id
        #print("text_content[0]",text_content[0])
        del text_content[0]
 

                       
        vec_sen_line=[0]*300     
        mass_sum=0    
        for word in text_content:        
            try:
                count_word=text_content.count(word)
                vec_sen_line=vec_sen_line+np.asarray(model[word]) / count_word  # center of gravity= ({Sigma pos(i)*mass(i)}/{Sigma mass(i)})
                mass_sum=mass_sum+1./count_word
            except:
                pass
                #print("UKN word",word)
                
        #print("vec_sen_line type",type(vec_sen_line))
        tf_word.append(np.asarray(vec_sen_line)/mass_sum)
     
    #print(text_id[0],tf_word[0])
    #print(len(text_id),len(tf_word))
    del text_id[0]
    del tf_word[0]
    print("file loaded!")
    file.close()
    return text_id,tf_word


filename='/home/miaohan/NLP/data/test.csv'
test_id,test_word=load_test(filename)

id [ 0.27929688 -0.08447266 -0.04150391  0.12011719 -0.01794434  0.12353516
  0.1328125  -0.08251953  0.11572266  0.10693359 -0.08056641 -0.01397705
 -0.13671875  0.11230469 -0.0546875   0.109375    0.12060547  0.06396484
 -0.02001953 -0.41992188  0.18652344  0.14550781 -0.08837891 -0.23828125
 -0.21386719 -0.24902344 -0.06884766  0.10693359  0.20996094  0.04589844
 -0.15527344 -0.29882812 -0.11376953 -0.02416992  0.01477051  0.01489258
 -0.06835938  0.33203125 -0.10888672 -0.14160156  0.05639648  0.05078125
  0.19921875  0.40820312  0.04833984  0.07714844  0.06689453 -0.171875
 -0.28515625 -0.203125   -0.2109375  -0.18164062  0.03369141 -0.21777344
  0.25976562  0.15234375  0.13964844 -0.22167969 -0.1484375   0.03808594
  0.02990723 -0.10644531 -0.10595703  0.11181641 -0.02770996  0.19726562
 -0.28320312  0.08886719  0.06347656  0.21386719  0.16308594  0.08300781
 -0.03088379  0.09716797  0.10253906 -0.17089844  0.22265625  0.38085938
  0.23535156 -0.10839844  0.08642578  0.06494141  

In [285]:
def train_test(text_author,tf_word):
    dindex=18000
    train_author,train_word=text_author[0:dindex],tf_word[0:dindex]
    test_author,test_word=text_author[dindex:],tf_word[dindex:]

    return train_word,train_author,test_word,test_author
a=[]
a.append(['Then', 'I', 'told', 'him', 'what', 'I'])
a.append(['scars', 'on', 'my', "ancestor", 'chest'])


train_x,train_y,test_x,test_y = train_test(text_author,tf_word)
print(len(train_x))
print(len(test_x))
#print(sen_vec)
#print(len(sen_vec))
#print(sen_vec[0][0])



    

18000
1579


In [286]:
def save_file(filename,tid,prediction):
    import csv
    with open(filename,"w") as csvfile: 
        writer = csv.writer(csvfile)

        #columns_name
        writer.writerow(["id","EAP","HPL","MWS"])
        #写入多行用writerows
        for ii in range(len(tid)):
            type(prediction[ii])
            a=[tid[ii]]+list(prediction[ii])
            writer.writerow(a)
            
            

In [287]:
from __future__ import print_function 
import tensorflow as tf

#mnist = input_data.read_data_sets("mnist_data/", one_hot=True) 

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 20
display_step = 1
print (len(text_id))  #19579

###### tf Graph Input #####
x = tf.placeholder(tf.float32, [None, 300]) # word2vec of shape 300 features
y = tf.placeholder(tf.float32, [None, 3]) # 3 authors to be classified => 3 classes

##### Set model weights #####
W = tf.Variable(tf.zeros([300, 3]))
b = tf.Variable(tf.zeros([3]))
 
 ##### Construct model #####
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
print(pred.get_shape())

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

##### Gradient Descent #####
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

##### Initializing the variables #####
init = tf.global_variables_initializer()


19579
(?, 3)


In [288]:
##### Launch the graph #####
with tf.Session() as sess:
    sess.run(init)
 
     ### Training cycle ###
    for epoch in range(training_epochs):
            
        avg_cost = 0.
        total_batch = int(len(train_x)/batch_size)
        print("epoch",epoch)
        print("total_batch",total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = train_x[i*batch_size:(i+1)*batch_size], train_y[i*batch_size : (i+1)*batch_size]
              # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                           y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
         # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
 
    print("Optimization Finished!")

    
    ##### Test model#####
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
    print(pred.get_shape())
    print(y.get_shape())
    ##### Calculate accuracy #####     
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: test_x, y: test_y}))
    print("submission", pred.eval({x:test_word}))
    prediction=pred.eval({x:test_word})
    #prediction.get_shape()
    filename='submission.csv'
    save_file(filename,test_id,prediction)

epoch 0
total_batch 900
Epoch: 0001 cost= 0.828629934
epoch 1
total_batch 900
Epoch: 0002 cost= 0.737129878
epoch 2
total_batch 900
Epoch: 0003 cost= 0.715645902
epoch 3
total_batch 900
Epoch: 0004 cost= 0.705181956
epoch 4
total_batch 900
Epoch: 0005 cost= 0.699023883
epoch 5
total_batch 900
Epoch: 0006 cost= 0.695026120
epoch 6
total_batch 900
Epoch: 0007 cost= 0.692269420
epoch 7
total_batch 900
Epoch: 0008 cost= 0.690289791
epoch 8
total_batch 900
Epoch: 0009 cost= 0.688826217
epoch 9
total_batch 900
Epoch: 0010 cost= 0.687720174
epoch 10
total_batch 900
Epoch: 0011 cost= 0.686869795
epoch 11
total_batch 900
Epoch: 0012 cost= 0.686206786
epoch 12
total_batch 900
Epoch: 0013 cost= 0.685683808
epoch 13
total_batch 900
Epoch: 0014 cost= 0.685267211
epoch 14
total_batch 900
Epoch: 0015 cost= 0.684932486
epoch 15
total_batch 900
Epoch: 0016 cost= 0.684661505
epoch 16
total_batch 900
Epoch: 0017 cost= 0.684440642
epoch 17
total_batch 900
Epoch: 0018 cost= 0.684259516
epoch 18
total_batch